# Transit agencies by Legislative Districts

* Get list of agencies associated with each senate district and assembly district. Use this [legislative district file.](https://gis.data.ca.gov/maps/cabaddc34c854421b38b8a9239315d9b/about)
* Combine districts with current members with dictionary and get list for Gillian.

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

import geopandas as gpd
import pandas as pd

from segment_speed_utils import helpers
from shared_utils import rt_dates
from legislative_dict import *

In [2]:
BASE_URL = ("https://services3.arcgis.com/fdvHcZVgB2QSRNkL/"
            "arcgis/rest/services/Legislative/FeatureServer/"
           )
ASSEMBLY_DISTRICTS = f"{BASE_URL}0/query?outFields=*&where=1%3D1&f=geojson"
SENATE_DISTRICTS = f"{BASE_URL}1/query?outFields=*&where=1%3D1&f=geojson"

In [3]:
# Import assembly districts and senate districts
assembly_districts = gpd.read_file(ASSEMBLY_DISTRICTS)[
    ["AssemblyDistrictLabel", "geometry"]]

senate_districts = gpd.read_file(SENATE_DISTRICTS)[
    ["SenateDistrictLabel", "geometry"]]

In [4]:
# Grab July's shapes and merge in operator name
analysis_date = rt_dates.DATES["jul2023"]

trips = helpers.import_scheduled_trips(
    analysis_date,
    columns = ["name", "shape_array_key"],
    get_pandas = True
)

trips = trips.assign(
    name = trips.name.str.replace(" Schedule", "")
)

shapes = helpers.import_scheduled_shapes(
    analysis_date,
    columns = ["shape_array_key", "geometry"],
    get_pandas = True,
    crs = "EPSG:4326"
).merge(
    trips,
    on = "shape_array_key",
    how = "inner"
)

In [5]:
def sjoin_shapes_to_legislative_geography(
    shapes: gpd.GeoDataFrame, 
    legislative_geog: gpd.GeoDataFrame
) -> pd.DataFrame:
    """
    Spatially join shapes to a legislative boundary.
    If at least one shape falls intersects with a district, include it.
    Create a df that's wide, so we can pull any assembly or senate district
    and grab a list of associated agencies.
    """
    district_col = [c for c in legislative_geog.columns 
                    if "DistrictLabel" in c][0]
    
    operators_by_district = (
        gpd.sjoin(
            shapes,
            legislative_geog,
            how = "inner",
            predicate = "intersects"
        )[["name", district_col]]
        .drop_duplicates()
        .sort_values([district_col, "name"])
        .reset_index(drop=True)
    )
    
    # Make wide, and have the operator names come out alphabetically
    wide_df = operators_by_district.groupby(
        district_col).name.agg(
        lambda x: ', '.join(x)
    ).reset_index().rename(columns = {district_col: "district"})
    
    return wide_df

In [6]:
senate_agencies = sjoin_shapes_to_legislative_geography(
    shapes, senate_districts
)

assembly_agencies = sjoin_shapes_to_legislative_geography(
    shapes, assembly_districts
)           

legislative_agencies = pd.concat(
    [senate_agencies, assembly_agencies], 
    sort = False
).reset_index(drop=True)

# Combine into 1 dict - read from legislative_dict
MEMBERS = {**SENATORS, **ASM}

In [7]:
# Put this into a spreadsheet format 
# Just grab the members Gillian asked for and attach name to district
df = legislative_agencies[
    legislative_agencies.district.isin(MEMBERS.values())
].reset_index(drop=True)

df = df.assign(
    member = df.district.map({v:k for k,v in MEMBERS.items()})
).rename(columns = {"name": "transit_agencies"}).reindex(
    columns = ["member", "district", "transit_agencies"])

In [8]:
df.to_csv('transit_agencies_for_legislative_members.csv', index=False)